In [109]:
"""
generate train data, validation data, and test data
"""
import os
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.ndimage import imread
from scipy.misc import imsave
from scipy.misc import imresize
import numpy as np
import os
from numpy import genfromtxt
import tensorflow as tf
from PIL import Image
import pickle
import helper
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import collections
import time

# Create Train and Target Data

In [7]:
# get codes
gallery_features_batch1 = np.load(open(r'gait_data/fc8_90/fc8_90_gallery_codes', mode='rb'))
probe_features_batch1 = np.load(open(r'gait_data/fc8_90/fc8_90_probe_codes', mode='rb'))
# feature shape
print("gallery length: {}".format(gallery_features_batch1.shape))
print("gallery shape: {}".format(gallery_features_batch1[0].shape))
print("")
print("probe length: {}".format(probe_features_batch1.shape))
print("probe shape: {}".format(probe_features_batch1[0].shape))

gallery length: (93,)
gallery shape: (32, 1000)

probe length: (93,)
probe shape: (31, 1000)


### Add Zero

In [37]:
gallery_features_batch1_add0 = np.zeros([len(gallery_features_batch1), 43, 1000], dtype=float)
probe_features_batch1_add0 = np.zeros([len(probe_features_batch1), 43, 1000], dtype=float)

g_seq_length = []
p_seq_length = []
for ii, (gallery, probe) in enumerate(zip(gallery_features_batch1, probe_features_batch1)):
    gallery_length = gallery.shape[0]
    probe_length = probe.shape[0]
    g_seq_length.append(gallery_length)
    p_seq_length.append(probe_length)
    
    gallery_features_batch1_add0[ii, :gallery_length, :] = gallery
    probe_features_batch1_add0[ii, :probe_length, :] = probe

### Create Target 1 Data

In [10]:
batch_1_target_1 = np.concatenate((gallery_features_batch1_add0, probe_features_batch1_add0), axis=1)
target_1 = np.ones([batch_1_target_1.shape[0]])

In [11]:
print("batch_1_target_1:", batch_1_target_1.shape)
print("")
print("target_1:", target_1.shape)

batch_1_target_1: (93, 86, 1000)

target_1: (93,)


### Create Target 0 Data
We create a same length batch with shuffled data
直到检测shuffle的idx没有重叠，函数会返回ok，否则打印error

In [20]:
idx = np.arange(len(gallery_features_batch1_add0))
np.random.shuffle(idx)
idx

array([28, 90, 79, 44, 41, 16, 38, 52, 31, 36, 78, 51, 73, 49, 25,  7, 12,
       57, 62, 17, 80, 55, 48, 74, 14, 89,  6, 40, 42, 37,  5,  0, 81,  9,
        2, 75, 43, 58, 54,  4, 32, 77, 34, 56, 82, 88, 76,  8, 61, 53, 23,
       72, 15, 35, 70, 68, 50, 29, 60, 67, 20, 30, 66, 11, 69, 71, 84, 85,
       18, 64, 47, 27, 92, 39, 87, 19, 10, 46, 45, 22, 33, 59,  1, 86, 83,
       13,  3, 24, 65, 91, 63, 21, 26])

In [21]:
def exam_shuffle_idx(idx):
    x =None
    for ii, index in enumerate(idx):
        if ii == index:
            x ='error'
            print(x)
            break
    if x != 'error':
        print("ok")
    return None
exam_shuffle_idx(idx)

ok


In [22]:
# create shuffled half_probe_features_batch1
shuffled_probe_features_batch1 = np.zeros(probe_features_batch1_add0.shape)
print(shuffled_probe_features_batch1.shape)
for ii, index in enumerate(idx):
    shuffled_probe_features_batch1[ii, :, :] = probe_features_batch1_add0[index]

(93, 43, 1000)


In [23]:
batch_1_target_0 = np.concatenate((gallery_features_batch1_add0, shuffled_probe_features_batch1), axis=1)
target_0 = np.zeros([batch_1_target_0.shape[0]])

In [24]:
batch_1_target_0[0][43:]

array([[ 0.14513677,  0.36747506,  0.29001927, ...,  0.12957594,
         0.35101527,  0.3333481 ],
       [ 0.11314621,  0.3523095 ,  0.27453646, ...,  0.09355041,
         0.36715832,  0.32119063],
       [ 0.08449331,  0.32923666,  0.31943917, ...,  0.13022023,
         0.36273172,  0.36481997],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [25]:
batch_1_target_1[28][43:]

array([[ 0.14513677,  0.36747506,  0.29001927, ...,  0.12957594,
         0.35101527,  0.3333481 ],
       [ 0.11314621,  0.3523095 ,  0.27453646, ...,  0.09355041,
         0.36715832,  0.32119063],
       [ 0.08449331,  0.32923666,  0.31943917, ...,  0.13022023,
         0.36273172,  0.36481997],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
# Concatenate half_batch_1_target_0 and half_batch_1_target_1 together
new_batch_1 = np.concatenate((batch_1_target_1, batch_1_target_0), axis=0)
new_batch_1.shape

(186, 86, 1000)

In [27]:
with open('gait_data/fc8_90/ordered_new_batch_1', 'wb') as f:
    np.save(f, new_batch_1)

In [29]:
target_1_and_0 = np.concatenate((target_1, target_0))

In [30]:
# Shuffle new_half_batch_1 to random order
idx2 = np.arange(len(new_batch_1))
np.random.shuffle(idx2)
idx2

array([ 78,  31,  59,  42,  68,  28, 180,  93, 162,  24,  49, 135,  47,
         7, 182,  40, 167,  43, 116, 169, 104,   6, 181,  62,  84, 145,
       138, 161, 185,  16, 105, 140, 121,  45, 110, 141, 125, 164, 152,
       132, 143,  71, 174,  46,   5, 112,  32, 156, 124,  76,  60, 160,
       113,  27, 147,  91, 111,  38,  29,  26, 114, 102,  70,  33,  12,
        10,  54, 165,  57,  25, 131,  92, 158, 144,   0,  82, 126,  17,
       146, 119, 148,  58,   4, 100, 133,  19, 170, 150, 115,  21,  67,
        94,  99,  48, 177, 101, 108,  98,  73,  83,  95,  20,  13, 106,
        34, 153,  14, 123,  18, 171,  81,  36,  22,  86, 134, 139, 184,
       151,   3, 129, 179,  97, 127, 107,   2, 157,  37, 172,  23, 178,
        96,  69, 149,  39,  77, 154, 103,  90,  44,  55,  65,  53,  11,
         9,   8,  56,  85,  35,  51, 166,  41,  52,  75,  89, 175, 173,
       142,   1, 176,  30,  79,  64, 168,  61, 120, 128, 130,  63,  80,
        50, 163,  15,  74, 117, 155, 183, 109,  66, 118,  88,  7

In [31]:
shuffled_new_batch_1 = np.ndarray(new_batch_1.shape, dtype=float)
shuffled_target_1_and_0 = np.ndarray(target_1_and_0.shape, dtype=float)
for ii, index in enumerate(idx2):
    shuffled_new_batch_1[ii, :, :] = new_batch_1[index]
    shuffled_target_1_and_0[ii] = target_1_and_0[index]

In [32]:
with open('gait_data/fc8_90/shuffled_new_batch_1', 'wb') as f:
    np.save(f, shuffled_new_batch_1)

In [33]:
one_hot_shuffled_new_batch_1_target = np.ndarray([186, 2], dtype=float)
for ii, each in enumerate(shuffled_target_1_and_0):
    if int(each) == 1:
        one_hot_shuffled_new_batch_1_target[ii, :] = 1, 0
    elif int(each) == 0:
        one_hot_shuffled_new_batch_1_target[ii, :] = 0, 1

In [34]:
with open('gait_data/fc8_90/one_hot_shuffled_new_batch_1_target', 'wb') as f:
    np.save(f, one_hot_shuffled_new_batch_1_target)

## Create seq_length

In [38]:
p_seq_length = np.array(p_seq_length)
shuffled_p_seq_length = np.zeros(p_seq_length.shape, dtype=int)
for ii, index in enumerate(idx):
    shuffled_p_seq_length[ii] = p_seq_length[index]

In [39]:
seq_column1 = np.reshape(np.concatenate([g_seq_length, g_seq_length]), [186, 1])
seq_column2 = np.reshape(np.concatenate([g_seq_length, shuffled_p_seq_length]), [186, 1])
seq_length = np.concatenate([seq_column1, seq_column2], axis=1)

shuffled_seq_length = np.zeros(seq_length.shape, dtype=int)
for ii, index in enumerate(idx2):
    shuffled_seq_length[ii] = seq_length[index]

In [40]:
with open('gait_data/fc8_90/shuffled_seq_length', 'wb') as f:
    np.save(f, shuffled_seq_length)

# Checkpoints 1

In [110]:
seq_length = np.load(open(r'gait_data/fc8_90/shuffled_seq_length', mode='rb'))
shuffled_new_batch_1 = np.load(open(r'gait_data/fc8_90/shuffled_new_batch_1', mode='rb'))
one_hot_shuffled_new_batch_1_target = np.load(open(r'gait_data/fc8_90/one_hot_shuffled_new_batch_1_target', mode='rb'))
print(shuffled_new_batch_1.shape)

(186, 86, 1000)


# Get Train and Validation data

In [111]:
print("data length: {}".format(len(shuffled_new_batch_1)))
print("every length has : {}".format(shuffled_new_batch_1.shape))

data length: 186
every length has : (186, 86, 1000)


In [112]:
split_frac = 0.8
split_num = int(split_frac*len(shuffled_new_batch_1))

train_x_raw, val_x_raw = shuffled_new_batch_1[:split_num], shuffled_new_batch_1[split_num:]
train_y_raw, val_y_raw = one_hot_shuffled_new_batch_1_target[:split_num], one_hot_shuffled_new_batch_1_target[split_num:]
train_seq_length, val_seq_length = seq_length[:split_num], seq_length[split_num:]

# print size
print("Train_x: {}".format(np.array(train_x_raw).shape))
print("Validation_x: {}".format(np.array(val_x_raw).shape))
print("Train_y: {}".format(np.array(train_y_raw).shape))
print("Validation_y: {}".format(np.array(val_y_raw).shape))
print("train_seq_length: {}".format(np.array(train_seq_length).shape))
print("val_seq_length: {}".format(np.array(val_seq_length).shape))

Train_x: (148, 86, 1000)
Validation_x: (38, 86, 1000)
Train_y: (148, 2)
Validation_y: (38, 2)
train_seq_length: (148, 2)
val_seq_length: (38, 2)


# According_len_get_outputs

In [113]:
def According_len_get_outputs(length_batch_1, length_batch_2, outputs_1, outputs_2):
    """
    length_batch_1 and length_batch_2 should minus one
    """
    new_outputs_1 = np.ndarray([outputs_1.shape[0], outputs_1.shape[2]], dtype=float)
    new_outputs_2 = np.ndarray([outputs_2.shape[0], outputs_2.shape[2]], dtype=float)
    
    for ii, (L1, L2, O1, O2) in enumerate(zip(length_batch_1, length_batch_2, outputs_1, outputs_2)):
        new_outputs_1[ii, :] = O1[L1, :]
        new_outputs_2[ii, :] = O2[L2, :]
        
    return new_outputs_1, new_outputs_2

# Build the graph

In [114]:
def build_rnn(batch_size=20, lstm_size=1000, num_layers=1, learning_rate=0.001, grad_clip=1):
    
    # reset graph
    tf.reset_default_graph()
    
    with tf.name_scope('inputs_1'):
        inputs_1 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_1")
    with tf.name_scope('inputs_2'):
        inputs_2 = tf.placeholder(tf.float32, [None, None, 1000], name="inputs_2")
    with tf.name_scope('targets'):
        target_ = tf.placeholder(tf.float32, [None, 2], name="targets")
        
    seq_1 = tf.placeholder(tf.float32, (None,), name="seq_1")
    seq_2 = tf.placeholder(tf.float32, (None,), name="seq_2")
    
    new_outputs_1 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_1")
    new_outputs_2 = tf.placeholder(tf.float32, [None, lstm_size], name="new_outputs_2")
    
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
#     seq_length = tf.placeholder(tf.int32, (None, ), name='seq_length')
    with tf.variable_scope("LSTM"):
#         with tf.name_scope("RNN_cells_1"):
            # Your basic LSTM cell
        #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
        #         lstm = tf.nn.relu(lstm)
        lstm_1 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_1 = tf.contrib.rnn.DropoutWrapper(lstm_1, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_1 = tf.contrib.rnn.MultiRNNCell([drop_1] * num_layers)

#         with tf.name_scope("RNN_init_state_1"):
            # Getting an initial state of all zeros
        initial_state_1 = cell_1.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_1"):
        outputs_1, final_state_1 = tf.nn.dynamic_rnn(cell_1, inputs_1, initial_state=initial_state_1, 
                                                     sequence_length=seq_1)

    with tf.variable_scope("LSTM", reuse=True):
#         with tf.name_scope("RNN_cells_2"):
            # Your basic LSTM cell
    #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
    #         lstm = tf.nn.relu(lstm)
        lstm_2 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_2 = tf.contrib.rnn.DropoutWrapper(lstm_2, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_2 = tf.contrib.rnn.MultiRNNCell([drop_2] * num_layers)

#         with tf.name_scope("RNN_init_state_2"):
            # Getting an initial state of all zeros
        initial_state_2 = cell_2.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_2"):
        outputs_2, final_state_2 = tf.nn.dynamic_rnn(cell_2, inputs_2, initial_state=initial_state_2, 
                                                     sequence_length=seq_2)

    with tf.name_scope('predictions'):
        # only choose final one
#         final_output_concat = tf.concat([outputs_1[:, -1], outputs_2[:, -1]], 1)

        # through the According_len_get_outputs
        final_output_concat = new_outputs_1 - new_outputs_2 
        logits = tf.contrib.layers.fully_connected(final_output_concat, 2)
        predictions = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', predictions)
        
    with tf.name_scope('cost'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_, logits=logits))
        tf.summary.scalar('cost', cost)
        
    with tf.name_scope('train'):
#         tvars = tf.trainable_variables()
#         grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip, name='clip')
#         train_op = tf.train.AdamOptimizer(learning_rate)
#         optimizer = train_op.apply_gradients(zip(grads, tvars))
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        
    with tf.name_scope('saver'):
        saver = tf.train.Saver()
        
    merged = tf.summary.merge_all()
    
    export_nodes = ['inputs_1', 'inputs_2', 'target_', 'initial_state_1', 'initial_state_2', 'outputs_1', 'outputs_2',
                    'final_state_1', 'final_state_2', 'keep_prob', 'cost', 'logits', 'predictions', 'optimizer', 
                    'merged', 'saver', 'learning_rate', 'cell_1', 'cell_2', 'final_output_concat', 'seq_1', 'seq_2',
                    'new_outputs_1', 'new_outputs_2']
    
    Graph = collections.namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Get Batches

In [115]:
def get_batches(data, target, seq_length, batch_size):
    n_batches = len(data)//batch_size
    data = data[:batch_size * n_batches]
    target = target[:batch_size * n_batches]
    for ii in range(0, batch_size*n_batches, batch_size):
        data_batch = data[ii:ii + batch_size]
        target_batch = target[ii:ii + batch_size]
        seq_length_batch_1 = seq_length[ii:ii + batch_size, 0]
        seq_length_batch_2 = seq_length[ii:ii + batch_size, 1]
        
        yield data_batch, target_batch, seq_length_batch_1, seq_length_batch_2

# Accuracy

In [116]:
train_seq_length

array([[28, 28],
       [31, 31],
       [30, 30],
       [33, 33],
       [30, 30],
       [31, 31],
       [30, 32],
       [32, 31],
       [26, 33],
       [32, 32],
       [27, 27],
       [33, 34],
       [32, 32],
       [31, 31],
       [31, 34],
       [32, 32],
       [31, 31],
       [30, 30],
       [33, 30],
       [30, 32],
       [32, 32],
       [35, 35],
       [30, 30],
       [29, 29],
       [31, 31],
       [23, 33],
       [25, 31],
       [30, 32],
       [28, 32],
       [33, 33],
       [30, 32],
       [32, 30],
       [31, 35],
       [25, 25],
       [31, 29],
       [29, 30],
       [32, 31],
       [31, 32],
       [30, 31],
       [31, 33],
       [28, 33],
       [31, 31],
       [31, 27],
       [31, 31],
       [31, 31],
       [30, 31],
       [32, 32],
       [30, 33],
       [31, 31],
       [30, 30],
       [30, 30],
       [31, 30],
       [30, 28],
       [32, 32],
       [29, 32],
       [34, 34],
       [31, 30],
       [33, 33],
       [29, 29

# Train

In [117]:
train_x = train_x_raw
val_x = val_x_raw
train_y = train_y_raw
val_y = val_y_raw

In [118]:
train_x.shape

(148, 86, 1000)

In [119]:
def train(model, epochs, file_writer, save_string):

    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        
        iteration = 1
        mean_val_Acc = 0
        mean_val_loss = 0
        count_Acc_not_increase_epochs = 0
        count_loss_not_decrease_epochs = 0
        Last_val_Acc = 0
        Last_val_loss = 0
        
        for e in range(epochs):
            for train_batch, target_batch, seq_length_batch_1, seq_length_batch_2 \
                             in get_batches(train_x, train_y, train_seq_length, batch_size):
                start = time.time()
#                 state_1 = sess.run(model.initial_state_1)
#                 state_2 = sess.run(model.initial_state_2)
#                 print(train_batch[:, :15, :].shape)
#                 print(target_batch)
    #             print('target: {}'.format(target_batch.shape))
                
                feed_1 = {
                        model.inputs_1: train_batch[:, :43, :],
                        model.inputs_2: train_batch[:, 43:, :],  
                        model.seq_1: seq_length_batch_1,
                        model.seq_2: seq_length_batch_2,
                        model.keep_prob: 1
                        }
            
#                 print(model.outputs_1.shape, model.outputs_1[:, -1], model.final_output_concat, 
#                       model.logits, model.predictions, model.cost)
                o_1, o_2 = sess.run([model.outputs_1, model.outputs_2], feed_dict=feed_1)
                
                real_outputs_1, real_outputs_2 = According_len_get_outputs(
                                               seq_length_batch_1 - 1, seq_length_batch_2 - 1, o_1, o_2)
#                 print(o_1[:, 28], o_2[:, 28])
#                 print(real_outputs_1, real_outputs_2)
                feed_2 = {
                        model.target_: target_batch,
#                         model.keep_prob: 1,
#                         model.initial_state_1: state_1,
#                         model.initial_state_2: state_2,
                        model.learning_rate: lr,
                        model.new_outputs_1: real_outputs_1,
                        model.new_outputs_2: real_outputs_2
                        }
                
                loss, _, logi, pred, summary,  = sess.run(
                                [model.cost, model.optimizer, model.logits, model.predictions, model.merged]
                                , feed_dict=feed_2)
                print(pred)
#                 print(logi)
#                 print('pred: {}'.format(type(pred)))
    #             print('pred: {}'.format(np.count_nonzero(pred)))
    #             print('pred: {}'.format(np.sum(pred>0.1)))
    #             print('pred: {}'.format(pred.size))
    #             print('output: {}'.format(out[1, 39, :]))
                
                if iteration%5==0:
                    end = time.time()
#                     acc = calculate_accuracy(sess, pred, target_batch, sq_length)
                    print("Epoch: {}/{},".format(e+1, epochs),' ',
                          "Iteration: {},".format(iteration),' ',
                          "Train loss: {:.3f},".format(loss),' ',
                          "{:.1f}s /batch.".format((end-start)/5),' '
#                           "ACCuracy: %{:.3f}".format(acc)
                         )
                    
                    file_writer.add_summary(summary, iteration)
                
#                 if iteration%25==0:
#                     validation_loss = []
#                     validation_Acc = []
                    
#                     if batch_size >= len(val_x):
#                         val_batch_size = 30
#                     else: 
#                         val_batch_size = batch_size
                    
#                     for val_batch, val_target_batch in get_batches(val_x, val_y, val_batch_size):

#                         val_state_1 = sess.run(model.cell_1.zero_state(val_batch_size, tf.float32))
#                         val_state_2 = sess.run(model.cell_2.zero_state(val_batch_size, tf.float32))

#                         feed = {model.inputs_1: val_batch[:, :15, :],
#                                 model.inputs_2: val_batch[:, 15:, :],
#                                 model.target_: val_target_batch,
#                                 model.keep_prob: 1,
#                                 model.initial_state_1: val_state_1,
#                                 model.initial_state_2: val_state_2,
#                                }

#                         val_loss, val_pred = sess.run([model.cost, model.predictions], feed_dict=feed)
                        
# #                         val_acc = calculate_accuracy(sess, val_pred, val_target_batch, val_sq_length)

#                         validation_loss.append(val_loss)
                        
# #                         validation_Acc.append(val_acc)
                        
#                     Last_val_Acc = mean_val_Acc
#                     Last_val_loss = mean_val_loss
                        
#                     mean_val_loss = sum(validation_loss)/len(validation_loss)  
#                     mean_val_Acc = sum(validation_Acc)/len(validation_Acc)
#                     print()
#                     print("Validation loss: {:.3f},".format(mean_val_loss), ' ',
#                           "Validation Accuracy: %{:.3f}".format(mean_val_Acc))
#                     print()
                    
                iteration += 1
                
            # Early stopping  
#             if mean_val_Acc - Last_val_Acc <= -0.3:
#                 count_Acc_not_increase_epochs += 1
#             if Last_val_loss - mean_val_loss <= -0.01:
#                 count_loss_not_decrease_epochs += 1
            
#             if count_Acc_not_increase_epochs >= 5:
#                 break
#             if count_loss_not_decrease_epochs >= 5:
#                 break
                
        model.saver.save(sess, r"{}".format(save_string))

In [120]:
!mkdir checkpoints_identification

mkdir: checkpoints_identification: File exists


In [ ]:
epochs = 1000
batch_size = 100
for lstm_size in [3000]:
    for num_layers in [1]:
        for lr in [0.0005]:
                log_string = 'logs/4/lr={},nl={},ls={},bs={}'.format(lr, num_layers, lstm_size, batch_size)
                save_string = 'checkpoints\lr={}_nl={}_ls={}_bs={}.ckpt'.format(lr, num_layers, lstm_size, batch_size)
                
                writer = tf.summary.FileWriter(log_string)

                model = build_rnn(batch_size=batch_size, lstm_size=lstm_size, 
                                  num_layers=num_layers, learning_rate=lr, grad_clip=100)

                train(model, epochs, writer, save_string)
                
                print(' ')
                print(' ')
                print("leraning_rate={},num_layers={},lstm_size={},batch_size={} finished, saved".format(lr, num_layers, lstm_size, batch_size))
                print(' ')
                print(' ')

[[ 0.50331169  0.49668837]
 [ 0.50471574  0.49528423]
 [ 0.49502885  0.50497115]
 [ 0.50416803  0.495832  ]
 [ 0.5010708   0.49892917]
 [ 0.50208986  0.49791008]
 [ 0.5         0.5       ]
 [ 0.49994349  0.50005656]
 [ 0.50161195  0.49838805]
 [ 0.49871805  0.50128192]
 [ 0.50395119  0.49604884]
 [ 0.5         0.5       ]
 [ 0.5013001   0.49869999]
 [ 0.5008375   0.4991625 ]
 [ 0.49892902  0.50107092]
 [ 0.49676579  0.50323421]
 [ 0.5055421   0.49445787]
 [ 0.50489622  0.49510387]
 [ 0.50210083  0.49789909]
 [ 0.5         0.5       ]
 [ 0.49999753  0.50000244]
 [ 0.5         0.5       ]
 [ 0.4956702   0.50432986]
 [ 0.50233489  0.49766511]
 [ 0.49982983  0.50017017]
 [ 0.51018447  0.4898155 ]
 [ 0.50201249  0.49798745]
 [ 0.50457138  0.49542868]
 [ 0.50178987  0.49821016]
 [ 0.50228655  0.49771348]
 [ 0.50425589  0.49574414]
 [ 0.50130916  0.49869078]
 [ 0.50733554  0.49266446]
 [ 0.49755993  0.50244009]
 [ 0.49661675  0.50338316]
 [ 0.49731573  0.50268418]
 [ 0.50109935  0.49890059]
 

[[ 0.52623051  0.47376946]
 [ 0.55510014  0.44489983]
 [ 0.56878102  0.43121892]
 [ 0.5         0.5       ]
 [ 0.49587435  0.50412565]
 [ 0.4950009   0.5049991 ]
 [ 0.49675289  0.5032472 ]
 [ 0.49255401  0.50744599]
 [ 0.49840936  0.50159061]
 [ 0.49097764  0.5090223 ]
 [ 0.49655941  0.5034405 ]
 [ 0.49617773  0.50382227]
 [ 0.49557513  0.50442487]
 [ 0.49278581  0.50721425]
 [ 0.48791578  0.51208419]
 [ 0.48615018  0.51384979]
 [ 0.5         0.5       ]
 [ 0.55403614  0.44596386]
 [ 0.48895371  0.51104629]
 [ 0.49291891  0.50708103]
 [ 0.49238324  0.50761676]
 [ 0.49312356  0.50687641]
 [ 0.48702702  0.51297295]
 [ 0.49257651  0.50742346]
 [ 0.49142584  0.50857413]
 [ 0.50010949  0.49989048]
 [ 0.49685752  0.50314248]
 [ 0.5         0.5       ]
 [ 0.49879041  0.50120962]
 [ 0.49299923  0.50700074]
 [ 0.5         0.5       ]
 [ 0.49450082  0.50549918]
 [ 0.5         0.5       ]
 [ 0.48971581  0.51028413]
 [ 0.48851234  0.51148766]
 [ 0.48842061  0.51157933]
 [ 0.49424011  0.50575989]
 

[[ 0.54798764  0.45201233]
 [ 0.59903544  0.40096462]
 [ 0.62782711  0.3721728 ]
 [ 0.49591956  0.50408041]
 [ 0.48810208  0.51189798]
 [ 0.48741531  0.51258475]
 [ 0.48961815  0.51038182]
 [ 0.48389745  0.51610255]
 [ 0.49172226  0.50827771]
 [ 0.48332319  0.51667678]
 [ 0.49082595  0.50917405]
 [ 0.48796532  0.51203471]
 [ 0.48769131  0.5123086 ]
 [ 0.48569238  0.51430762]
 [ 0.47928122  0.52071881]
 [ 0.47914168  0.52085835]
 [ 0.49471655  0.50528342]
 [ 0.59845275  0.40154728]
 [ 0.47981411  0.52018589]
 [ 0.48570174  0.51429826]
 [ 0.48361817  0.51638186]
 [ 0.48518622  0.51481384]
 [ 0.48022583  0.5197742 ]
 [ 0.48602873  0.51397127]
 [ 0.48340431  0.51659572]
 [ 0.5         0.5       ]
 [ 0.48900855  0.51099145]
 [ 0.49711722  0.50288278]
 [ 0.49136889  0.50863105]
 [ 0.48644811  0.51355195]
 [ 0.49277273  0.50722724]
 [ 0.4850933   0.5149067 ]
 [ 0.5         0.5       ]
 [ 0.4817802   0.51821977]
 [ 0.47995165  0.52004838]
 [ 0.47779822  0.52220178]
 [ 0.48679924  0.51320076]
 

[[ 0.56934661  0.43065342]
 [ 0.64057523  0.35942477]
 [ 0.68187112  0.31812891]
 [ 0.48776048  0.51223952]
 [ 0.4795824   0.52041763]
 [ 0.47861439  0.52138567]
 [ 0.48130837  0.5186916 ]
 [ 0.47436708  0.52563298]
 [ 0.48328635  0.51671368]
 [ 0.47493392  0.52506608]
 [ 0.4842304   0.51576954]
 [ 0.47909746  0.52090257]
 [ 0.47923765  0.52076232]
 [ 0.47807887  0.5219211 ]
 [ 0.47015756  0.5298425 ]
 [ 0.47227871  0.52772129]
 [ 0.48746884  0.5125311 ]
 [ 0.6405279   0.35947213]
 [ 0.47069728  0.52930272]
 [ 0.47780138  0.52219868]
 [ 0.4741312   0.52586877]
 [ 0.47694319  0.52305681]
 [ 0.473239    0.52676094]
 [ 0.47911999  0.52087998]
 [ 0.4752138   0.52478623]
 [ 0.49715009  0.50284994]
 [ 0.47977528  0.52022463]
 [ 0.48975202  0.51024795]
 [ 0.4826701   0.51732993]
 [ 0.47941977  0.52058017]
 [ 0.48361245  0.51638758]
 [ 0.47514668  0.52485335]
 [ 0.49419653  0.50580347]
 [ 0.4733991   0.5266009 ]
 [ 0.47103626  0.5289638 ]
 [ 0.46679744  0.53320253]
 [ 0.47821483  0.52178514]
 

[[ 0.59003484  0.40996519]
 [ 0.67907536  0.32092461]
 [ 0.72998905  0.27001092]
 [ 0.47839341  0.52160656]
 [ 0.47060221  0.52939779]
 [ 0.4689326   0.53106743]
 [ 0.47218779  0.52781218]
 [ 0.46448332  0.53551668]
 [ 0.47351265  0.52648741]
 [ 0.46608931  0.53391075]
 [ 0.47686392  0.52313614]
 [ 0.47016382  0.52983618]
 [ 0.47062314  0.52937686]
 [ 0.47006446  0.52993554]
 [ 0.46096122  0.53903872]
 [ 0.46562418  0.53437591]
 [ 0.47896633  0.5210337 ]
 [ 0.67953336  0.32046664]
 [ 0.46181726  0.53818274]
 [ 0.46951738  0.53048265]
 [ 0.46446121  0.53553879]
 [ 0.46863985  0.53136015]
 [ 0.46600196  0.53399807]
 [ 0.47189787  0.5281021 ]
 [ 0.46712652  0.53287351]
 [ 0.48806375  0.51193625]
 [ 0.46969539  0.53030455]
 [ 0.48133928  0.51866066]
 [ 0.47308624  0.52691382]
 [ 0.47193804  0.52806193]
 [ 0.47299427  0.52700573]
 [ 0.46514383  0.5348562 ]
 [ 0.48543224  0.51456779]
 [ 0.46493027  0.5350697 ]
 [ 0.46222052  0.53777957]
 [ 0.45615059  0.54384935]
 [ 0.4688158   0.53118426]
 

...

[[ 0.87219214  0.12780783]
 [ 0.97577548  0.02422456]
 [ 0.99234927  0.00765076]
 [ 0.36210772  0.63789225]
 [ 0.38540569  0.61459428]
 [ 0.33531821  0.66468179]
 [ 0.37138039  0.62861961]
 [ 0.34580699  0.65419298]
 [ 0.3355045   0.66449547]
 [ 0.37769759  0.62230241]
 [ 0.43565461  0.56434536]
 [ 0.39484408  0.60515589]
 [ 0.41515416  0.58484584]
 [ 0.40819308  0.59180689]
 [ 0.36708042  0.63291955]
 [ 0.46540076  0.53459924]
 [ 0.37468171  0.62531823]
 [ 0.9767909   0.02320909]
 [ 0.39051464  0.60948539]
 [ 0.39383897  0.60616106]
 [ 0.37057847  0.62942153]
 [ 0.40738463  0.59261537]
 [ 0.42192659  0.57807338]
 [ 0.44237053  0.55762953]
 [ 0.428058    0.57194197]
 [ 0.34808543  0.65191454]
 [ 0.3412838   0.65871614]
 [ 0.40683344  0.59316659]
 [ 0.35261771  0.64738226]
 [ 0.42574787  0.57425207]
 [ 0.30297339  0.69702661]
 [ 0.35285854  0.64714146]
 [ 0.35300758  0.64699244]
 [ 0.39937392  0.60062605]
 [ 0.39880407  0.60119593]
 [ 0.33214396  0.66785598]
 [ 0.35342702  0.64657295]
 

[[ 0.87544477  0.12455523]
 [ 0.97700197  0.02299811]
 [ 0.99285167  0.00714831]
 [ 0.3617045   0.63829559]
 [ 0.38511553  0.61488444]
 [ 0.33407801  0.66592193]
 [ 0.37123221  0.62876773]
 [ 0.34482953  0.65517038]
 [ 0.33431432  0.66568565]
 [ 0.37712011  0.62287986]
 [ 0.43637276  0.56362718]
 [ 0.3950648   0.60493517]
 [ 0.41595998  0.58403999]
 [ 0.40813777  0.59186226]
 [ 0.36628732  0.63371265]
 [ 0.46639678  0.53360319]
 [ 0.37361863  0.62638134]
 [ 0.97798342  0.02201656]
 [ 0.39021429  0.60978568]
 [ 0.39298105  0.60701901]
 [ 0.37078556  0.62921435]
 [ 0.40730193  0.59269804]
 [ 0.42146498  0.57853508]
 [ 0.44283694  0.55716306]
 [ 0.42850506  0.571495  ]
 [ 0.34705675  0.65294331]
 [ 0.34019125  0.65980875]
 [ 0.40708438  0.59291559]
 [ 0.35178971  0.64821023]
 [ 0.42577946  0.5742206 ]
 [ 0.30129299  0.69870704]
 [ 0.35202414  0.64797592]
 [ 0.35187465  0.64812541]
 [ 0.39908865  0.60091132]
 [ 0.39927986  0.60072011]
 [ 0.33082673  0.6691733 ]
 [ 0.35257393  0.64742607]
 

[[ 0.87856764  0.12143236]
 [ 0.97814548  0.02185449]
 [ 0.9933123   0.00668767]
 [ 0.36144832  0.63855171]
 [ 0.38495469  0.61504531]
 [ 0.33296749  0.66703248]
 [ 0.3712323   0.62876773]
 [ 0.34398347  0.65601653]
 [ 0.33327824  0.6667217 ]
 [ 0.37667561  0.62332439]
 [ 0.43724123  0.5627588 ]
 [ 0.39540637  0.60459369]
 [ 0.41688684  0.58311313]
 [ 0.40821236  0.59178758]
 [ 0.36561355  0.63438642]
 [ 0.46749818  0.53250182]
 [ 0.37269899  0.62730104]
 [ 0.97909456  0.02090549]
 [ 0.39001873  0.60998124]
 [ 0.39224538  0.60775465]
 [ 0.37111139  0.62888861]
 [ 0.4073379   0.5926621 ]
 [ 0.42112887  0.57887113]
 [ 0.44344079  0.55655921]
 [ 0.42907977  0.57092023]
 [ 0.3462151   0.65378487]
 [ 0.33924571  0.66075432]
 [ 0.40747812  0.59252191]
 [ 0.35109594  0.64890409]
 [ 0.42594492  0.57405508]
 [ 0.29975873  0.70024133]
 [ 0.3513006   0.6486994 ]
 [ 0.35089725  0.64910275]
 [ 0.39892614  0.6010738 ]
 [ 0.3998774   0.60012263]
 [ 0.32961965  0.67038041]
 [ 0.35185993  0.64814007]
 

[[ 0.88156801  0.11843202]
 [ 0.9792133   0.02078675]
 [ 0.99373537  0.00626461]
 [ 0.36131087  0.6386891 ]
 [ 0.38489583  0.61510414]
 [ 0.33196068  0.66803926]
 [ 0.37135378  0.62864625]
 [ 0.34324369  0.65675634]
 [ 0.33236799  0.66763204]
 [ 0.37633806  0.62366199]
 [ 0.438232    0.56176794]
 [ 0.39584336  0.60415661]
 [ 0.41790786  0.58209205]
 [ 0.40839058  0.59160942]
 [ 0.36503485  0.63496512]
 [ 0.46867999  0.53131998]
 [ 0.37189475  0.62810522]
 [ 0.98013103  0.01986894]
 [ 0.38990316  0.61009681]
 [ 0.39160648  0.60839355]
 [ 0.37152979  0.62847024]
 [ 0.40746728  0.59253275]
 [ 0.42089272  0.57910722]
 [ 0.44415513  0.55584484]
 [ 0.42975643  0.5702436 ]
 [ 0.34552699  0.65447301]
 [ 0.33842003  0.66157997]
 [ 0.40798482  0.59201521]
 [ 0.35050949  0.64949054]
 [ 0.42621756  0.5737825 ]
 [ 0.29834336  0.7016567 ]
 [ 0.35066336  0.64933658]
 [ 0.35004565  0.64995444]
 [ 0.39886099  0.60113895]
 [ 0.40057144  0.59942859]
 [ 0.32850012  0.67149991]
 [ 0.35125867  0.6487413 ]
 

[[ 0.8844524   0.11554762]
 [ 0.98021138  0.01978855]
 [ 0.99412459  0.00587542]
 [ 0.36126453  0.63873553]
 [ 0.38491201  0.61508799]
 [ 0.33103254  0.66896749]
 [ 0.37157008  0.62842989]
 [ 0.34258562  0.65741444]
 [ 0.33155572  0.66844428]
 [ 0.3760812   0.62391877]
 [ 0.43931675  0.56068331]
 [ 0.39635098  0.60364902]
 [ 0.41899651  0.58100349]
 [ 0.4086456   0.59135437]
 [ 0.36452702  0.63547295]
 [ 0.46991637  0.5300836 ]
 [ 0.37117791  0.62882209]
 [ 0.98109931  0.01890068]
 [ 0.38984287  0.61015707]
 [ 0.39103824  0.60896176]
 [ 0.37201551  0.62798446]
 [ 0.40766457  0.5923354 ]
 [ 0.42073002  0.57927001]
 [ 0.44495219  0.55504781]
 [ 0.43050873  0.56949127]
 [ 0.34495923  0.65504074]
 [ 0.33768752  0.66231245]
 [ 0.40857482  0.59142518]
 [ 0.35000399  0.64999598]
 [ 0.42656979  0.57343024]
 [ 0.297021    0.70297903]
 [ 0.35008854  0.64991146]
 [ 0.3492907   0.65070933]
 [ 0.39886755  0.60113239]
 [ 0.40133691  0.59866309]
 [ 0.32744625  0.67255372]
 [ 0.35074413  0.64925587]
 